[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

# Median NDVI Calculation Script

This script calculates the median NDVI for a given region and years of interest, based on Landsat 8 imagery (or you can adjust it for Sentinel-2). It then exports the results to your Google Drive.

In [ ]:
import ee
import folium

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [ ]:
# Define the region of interest from your asset.
roi = ee.FeatureCollection('projects/ee-komba/assets/bbox_wirong')

# Define the years of interest.
years = [2013, 2015, 2017, 2019, 2021, 2023]

# Define the date range for April-June.
start_month = '04-01'
end_month = '06-30'

In [ ]:
# Function to calculate median NDVI for a given year.
def calculate_median_ndvi(year):
    start_date = ee.Date.fromYMD(year, 4, 1)
    end_date = ee.Date.fromYMD(year, 6, 30)

    # Load Landsat 8 OLI Surface Reflectance.
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .map(lambda image: image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')) \
        .select('NDVI')

    # Calculate the median NDVI.
    median_ndvi = collection.median()

    # Clip to the region of interest.
    median_ndvi = median_ndvi.clip(roi)

    # Export the image to Google Drive
    export_task = ee.batch.Export.image.toDrive(
        image=median_ndvi,
        description=f'Median_NDVI_{year}',
        folder='NDVI_Results',  # This folder will be created in your Google Drive
        fileNamePrefix=f'median_NDVI_{year}',
        region=roi.geometry(),
        scale=100,  # 100m scale as requested
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )

    export_task.start()

    return median_ndvi.set('year', year)  # Return the image with year property.

In [ ]:
# Map the function over the years and get a list of images.
median_ndvi_images = [calculate_median_ndvi(year) for year in years]

In [ ]:
# Initialize the map centered on the region of interest.
my_map = folium.Map(location=[-7.7, 15.4], zoom_start=7)  # Adjust center to your AOI

# Add each year's median NDVI to the map.
for image in median_ndvi_images:
    year = image.get('year').getInfo()
    vis_params = {
        'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']
    }
    url = image.getMapId(vis_params)['tile_fetcher'].url_format
    folium.TileLayer(tiles=url, attr=f'Median NDVI {year}').add_to(my_map)

# Display the map.
my_map